# EXPLORATION 4: Lyricist

EX4 정도 진행을 하니 코딩의 기록을 어떻게 해야 하는지 좀 틀이 잡히는 듯 하다.  
프로그래밍을 진행하면서 나의 의식의 흐름으로 작성을 하는것이 가장 좋을 듯 하다.  
내가 무엇을 몰랐고, 무엇을 알려고 노력했으며, 어떤식으로 이해하고 넘어 갔는지를 기록해 볼 수 있다.  
목표한 결과가 나오기까지의 전체적인 과정들을 알 수 있을 것이다.

## (1) 패키지들을 import 하자

In [1]:
# 언제나 그렇듯 사용할 패키지 a.k.a 모듈 또는 라이브러리들을 불러 온다.

import glob    # txt파일 읽어오는 glob이라는 패키지를 사용해 보도록 하자.
import os, re 
import numpy as np
import tensorflow as tf

## (2) 데이터 파일을 읽어들이자 (오늘의 데이터는 텍스트 데이터)

변하지 진리, 좋은 데이터는 좋은 모델을 만든다.  
데이터를 잘 이해하고 다듬는 것은 좋은 모델을 만드는 첫 걸음이다.
자 그러면 일단 데이터 파일부터 읽어들이자, 시작이 반이다.

- ### 폴더에 존재하는 모든 텍스트 파일의 리스트를 읽어들이자.

(0) 한동안 이미지 파일만 가지고 프로젝트를 했더니 일반 파읽 읽기가 좀 생소하게 느껴진다.  
(1) corpus는 '말뭉치'라는 표현이다. raw_corpus에 방금불러들인 날것의 '말뭉치'들(텍스트 데이터)들을 저장하겠다는 의미  
(2) 폴어 안에 있는 모든 파일의 경로를 glob을 사용하여 읽어들인다.

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/workplace/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = [] # 모든 것은 이 변수로 부터 시작된다. 집중하자, 여기에 raw data를 저장한다.

- ### 일단 다음으로 넘어가기 전에 슬쩍 뭐가 들어있는지, 데이터 타입은 무엇인지 확인부터 하고 넘어가자

In [3]:
# test list 한번 확인해 보고 넘어가자 무엇이 들었을까?

print(type(txt_list)) # 데이터 타입도 확인해주고, 예상한대로 리스트형
txt_list

<class 'list'>


['/aiffel/aiffel/workplace/lyricist/data/lyrics/leonard-cohen.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/lil-wayne.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/blink-182.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/bruce-springsteen.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/notorious_big.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/beatles.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/bob-dylan.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/alicia-keys.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/dr-seuss.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/radiohead.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/nickelback.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/prince.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/joni-mitchell.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/missy-elliott.txt',
 '/aiffel/aiffel/workplace/lyricist/data/lyrics/dj-khaled.txt',
 '/aiffel/aiffel/wo

- ### 텍스트 파일들의 내용을 모두 읽어 raw_corpus에 저장

LMS에서 실습한 것은 텍스트 파일 하나에 담긴 내용이라 읽은 후 바로 raw_corpus에 담을 수 있었지만  
지금은 여러개의 파일에서 내용을 모두 읽어들여야 한다.  
그래서 일단 데이터 폴더에 존재하는 텍스트 파일들의 제목을 모두 읽어서 그것을 리스트 (txt_list)로 저장하고  
그 리스트를 기반으로 각 파일의 내용을 읽어서 raw_corpus에 저장 하도록 코딩하면 된다.

In [4]:
# 여러개의 txt 파일의 내용을 모두 읽어서 raw_corpus에 담는다.
# close를 빼먹는 것을 방지하기 위해서 항상 파일을 오픈 할때는 with open() as X 라는 표현을 쓰는 것을 습관하 하도록 하자.

for txt_file in txt_list:
    with open(txt_file, "r") as f:  # f 로 부르자
        raw = f.read().splitlines() # 한줄씩 읽어들이자 --> splitline()
        raw_corpus.extend(raw)      # 결국 txt_list에 있는 모든 파일들을 다 읽어 들여서 raw_corpus에 iterable 자료형인 리스트 형으로 저장하라는 의미

- ### 로딩된 텍스트 데이터 내용 확인

(1) 잘 불러들여 졌는지 확인하기 위해서 앞에 몇개만 출력해 본다.  
(2) 잘 불러 들여진듯 하니.... 일단 다음으로 패스

In [5]:
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [86]:
raw_corpus # 전체 데이터도 한번 확인해 보고, Hallelujah는 제거해야 하는게 아닐까 하는 생각이든다. 데이터가 너무 길게 출력되어 실행결과는 생략

In [37]:
print(raw_corpus[:9]) # 대신 테스트로 앞에서 9개만 출력해본다

["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah']


## (3) 텍스트 데이터 필터링

### 문장 정제 함수 정의

LMS 샘플과는 상황이 다르다
데이터를 확인해보면 기존의 규칙에서 추가 되어야 하는 부분이 있다는 것을 확인할 수 있다.  
예를들면 I've don't won't 와 같은 것은 한 단어로 만들어 줘야 한다.  
하지만 LMS에서 사용한 기존의 규칙을 그대로 복사해서 사용하면 '도 특수문자로 취급하기 때문에  
양쪽에 공백을 만들고 I ve don t won t 이런식으로 단어를 분리해서 이상한 단어를 만들어 버린다.  
해당 단어가 손상되지 않고 정확하게 남도록 코드를 수정해야 한다

- 원하는 데이터를 얻기 위해서 어떤 형태로 데이터를 가공 또는 필터링 해야 하는지 확인해야 한다. 필터링 해야 하는 문장이나 단어들의 특징을 찾아 낸다.  

- enumerate() 리스트나 문자열의 index값을 출력하고 싶을경우에 사용한다.  
- for문의 경우에는 이 인덱스 값을 사용해야 하므로 enumerate를 사용한다.  
- 우리는 문자열을 사용하고 있기 때문에 enumerate를 사용하여 반복문을 돌리면 된다.  

In [46]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?'.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다. 여기에서 ' 기호를 추가 하였다.
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?'.!,¿]+", " ", sentence) # 4 '를 추가하여 위에서 언급한 단어들을 보존하였다. 
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

### 정제된 문장을 모으자

In [47]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    #if len(sentence) == 0: continue
    #if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

["<start> now i've heard there was a secret chord <end>",
 '<start> that david played , and it pleased the lord <end>',
 "<start> but you don't really care for music , do you ? <end>",
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

#### 정규표현식을 사용해서 이상한 단어를 찾아서 바꾸기 작업을 하려고 하는데 일단 코드가 어려워서 나중에 작업예정

전체 데이터 확인중에 You?d 과 같은 문장이 보인다. You'd 같은데 뭔가 잘못된 것이 있는듯 하다.  
이런 것들이 몇개 보인다. 해당 형식을 가지고 있는 단어를 찾아서 정상적으로 바꾸는 코드를 만들려고 하는데 아직 구현을 못했다.  
일단은 그대로 사용하여 학습 시킬예정

In [87]:
#find_word = 'You?d'

#result = [i for i in range(len(raw_corpus)) if find_word in raw_corpus[i]]
#print(result)

In [85]:
#p = re.compile('[a-zA-Z]+[?][a-zA-Z]+')
#m = p.match(raw_corpus[202])


#result = [i for i in range(len(raw_corpus)) if find_word in raw_corpus[i]]
#result = [i for i in range(len(raw_corpus)) if p in raw_corpus[i]]
#print(result)

### Tokenizer 함수

maxlen을 설정하지 않고 가장 긴문장으로 맞추는 디폴트 옵션으로 설정하면 1 epoch당 47분의 학습시간이 걸린다.  
총 10 epoch를 돌리게 되면 470분의 약 8시간이 걸린다.  
이는 상황상 무리가 있기 때문에 maxlen을 15로 설정하고 프로젝트를 진행하였다.  

전체단어의 수를 12,000개로 변경하였고 maxlen은 15로 설정하였다,

In [73]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,   # 전체 단어의 개수 
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   46  149 ...    0    0    0]
 [   2   15 2768 ...    0    0    0]
 [   2   32    7 ...    3    0    0]
 ...
 [   2  129   20 ...   10 1040    3]
 [8545    6   35 ... 1312  656    3]
 [   2    7   30 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fcedaf3f5e0>


In [75]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2   46  149  302   86   53    9 1006 6212    3    0    0    0    0]
[  46  149  302   86   53    9 1006 6212    3    0    0    0    0    0]


In [76]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [77]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (149670, 14)
Target Train: (149670, 14)


In [78]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512 # 워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기 
hidden_size = 1024   # 얼마나 많은 일꾼을 둘 것인가?
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

print("ok")

ok


In [89]:
# the save point
checkpoint_dir = os.getenv('HOME')+'/aiffel/workplace/lyricist/models/lyricist'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

### 훈련결과 loss = 1.9534

In [80]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
730/730 [==============================] - 141s 188ms/step - loss: 3.4005
Epoch 2/10
730/730 [==============================] - 140s 191ms/step - loss: 2.9567
Epoch 3/10
730/730 [==============================] - 141s 192ms/step - loss: 2.7652
Epoch 4/10
730/730 [==============================] - 140s 191ms/step - loss: 2.6166
Epoch 5/10
730/730 [==============================] - 140s 192ms/step - loss: 2.4861
Epoch 6/10
730/730 [==============================] - 141s 192ms/step - loss: 2.3669
Epoch 7/10
730/730 [==============================] - 140s 192ms/step - loss: 2.2558
Epoch 8/10
730/730 [==============================] - 140s 192ms/step - loss: 2.1503
Epoch 9/10
730/730 [==============================] - 141s 192ms/step - loss: 2.0499
Epoch 10/10
730/730 [==============================] - 141s 192ms/step - loss: 1.9534


In [81]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
        tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.
print("ok")

ok


### good을 넣고 돌려본 결과는 다음과 같이 나왔다.

### good morning , give me a million reasons

In [87]:
generate_text(model, tokenizer, init_sentence="<start> good", max_len=20)

'<start> good morning , give me a million reasons <end> '

## 회고

이번 EX04는 지금까지 진행한 프로젝트중에서 제일 까다로웠다.  

1. 데이터의 특성도 최대한 자세하게 고려하기 위해서 까다롭게 살펴 보는 과정에 공을 많이 들였다.  
이때 I've don't와 같은 특성을 발견해서 손상되지 않도록 필터링을 하였다. 
또한 문자들중 You?d 와 같이 표현된 다수의 문자들을 발견해서 정규표현식을 사용하여 You'd와 같이 수정하려고 했지만 해당 부분은 조금 더 공부를 해야 할 것 같다는 판단에 일단 중지하고 프로젝트를 계속 진행하였다. 정규표현식을 사용해서 특정한 조건의 문자를 반복적으로 교체하는 부분에 대한 이해가 필요하다.


2. 사실은 모든 데이터를 사용하여 모델을 만들기 위해서. Tokenizer() 함수에서 max_len을 디폴트 값으로 설정하고 훈련을 했으나 시간이 너무 오래 걸리는 상황이 되었다. 다른 변수들을 수정하여도 해당 시간은 크게 달라지지 않았다. 하여 max_len을 15로 변경한 상태에서 훈련을 진행하였다. 추후에 이부분에 대해서는 다시한번 확인하고 점검하는 것이 필요하다고 생각된다. 


3. 해당 모델에 대한 이론이 부족한 상태에서 프로젝트를 진행하니 역부족이라는 생각이 들었다. 일단은 큰 그림을 먼저 본다고 생각하고 무리 없이 작동하는 것에 촛점을 맞추어 진행하였다. 


4. 데이터 전처리에 대한 기능적인 부분 방법론 적인 부분에 대한 심도있는 공부가 추가적으로 필요할 듯 싶다.

## Study for EX4

이곳은 EX4를 진행하면서 필요한 공부를 진행한 흔적을 남기는 공간입니다.  
추후 내가 무엇을 몰랐고 무엇을 헛갈려 했는지 확인할 수 있습니다.
기록은 실력입니다.

### splitlines() 함수 테스트

In [9]:
traveler = """강나루 건너서\n밀밭 길을\n\n구름에 달 가듯이\n가는 나그네\n

길은 외줄기\n남도 삼백리\n\n술 익는 마을마다\n타는 저녁놀\n

구름에 달 가듯이\n가는 나그네"""

In [10]:
poet = traveler.splitlines()

In [12]:
print(poet)

['강나루 건너서', '밀밭 길을', '', '구름에 달 가듯이', '가는 나그네', '', '', '길은 외줄기', '남도 삼백리', '', '술 익는 마을마다', '타는 저녁놀', '', '', '구름에 달 가듯이', '가는 나그네']


In [11]:
for line in poet:

    print(line)

강나루 건너서
밀밭 길을

구름에 달 가듯이
가는 나그네


길은 외줄기
남도 삼백리

술 익는 마을마다
타는 저녁놀


구름에 달 가듯이
가는 나그네


### 정제 함수

In [90]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [20]:
import numpy as np
import tensorflow as tf

x = np.random.sample((10, 4))
print(x)

dataset = tf.data.Dataset.from_tensor_slices(x)
#type(dataset)

for i in dataset:
    print(i)

[[0.45265942 0.46149126 0.66491722 0.57913661]
 [0.84503418 0.61600919 0.09576326 0.88066692]
 [0.18162063 0.35083383 0.64304336 0.99273951]
 [0.63371814 0.8565983  0.20265233 0.49946737]
 [0.09757024 0.0536169  0.82579491 0.16102758]
 [0.48184313 0.40398499 0.6522637  0.59592203]
 [0.11996699 0.43222936 0.27585619 0.89409328]
 [0.98788334 0.83711884 0.16414894 0.61062295]
 [0.66037826 0.26647478 0.05190215 0.56254569]
 [0.34664122 0.55441537 0.86177822 0.95355645]]
tf.Tensor([0.45265942 0.46149126 0.66491722 0.57913661], shape=(4,), dtype=float64)
tf.Tensor([0.84503418 0.61600919 0.09576326 0.88066692], shape=(4,), dtype=float64)
tf.Tensor([0.18162063 0.35083383 0.64304336 0.99273951], shape=(4,), dtype=float64)
tf.Tensor([0.63371814 0.8565983  0.20265233 0.49946737], shape=(4,), dtype=float64)
tf.Tensor([0.09757024 0.0536169  0.82579491 0.16102758], shape=(4,), dtype=float64)
tf.Tensor([0.48184313 0.40398499 0.6522637  0.59592203], shape=(4,), dtype=float64)
tf.Tensor([0.11996699 0.4

In [24]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])

for i in dataset:
    print(i)

tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)


In [25]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))

for i in dataset1:
    print(i)

tf.Tensor(
[0.05728447 0.7002839  0.3504312  0.16974461 0.08753824 0.49075675
 0.034549   0.68586886 0.6436845  0.61786485], shape=(10,), dtype=float32)
tf.Tensor(
[0.7744615  0.7309557  0.34500647 0.00230932 0.9476961  0.3632636
 0.6413373  0.739557   0.21821117 0.4731332 ], shape=(10,), dtype=float32)
tf.Tensor(
[0.5883421  0.62269795 0.7505076  0.7154732  0.77434456 0.08255303
 0.27171493 0.07187235 0.24762654 0.12759483], shape=(10,), dtype=float32)
tf.Tensor(
[0.37824535 0.85496557 0.37605762 0.43130946 0.9210603  0.2741623
 0.0866679  0.96838987 0.29742897 0.89832604], shape=(10,), dtype=float32)


### strip() 함수에 대한 이해

In [12]:
ex_str = "                   hello         "
ex_str.strip()

'hello'

In [16]:
' Now I heard there was a secret chord '.strip()

'Now I heard there was a secret chord'

In [21]:
result = re.sub(r"([?.!,¿])", r" \1 ", "But you don't really care for music, do you?")
print(result)

But you don't really care for music ,  do you ? 


### batch size가 대체 무엇인지몰라서 찾아본 자료

https://m.blog.naver.com/qbxlvnf11/221449297033